In [1]:
import os
import folium 
from folium import plugins
from branca.element import Template, MacroElement

In [2]:
import json 
import pandas as pd
import requests
from pandas.io.json import json_normalize


In [3]:
states = pd.read_json("https://gist.githubusercontent.com/meiqimichelle/7727723/raw/0109432d22f28fd1a669a3fd113e41c4193dbb5d/USstates_avg_latLong")

states.head()

,state,latitude,longitude
0,Alaska,61.3850,-152.2683
1,Alabama,32.7990,-86.8073
2,Arkansas,34.9513,-92.3809
3,Arizona,33.7712,-111.3877
4,California,36.1700,-119.7462


In [4]:
state_abv = pd.read_json("https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_titlecase.json")

state_abv.head()

,name,abbreviation
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [5]:
# Combine state data with state abreviation, but first modify column name shared between the two

state_abv = state_abv.rename(columns={"name":"state"})

state_abv.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [6]:
states = pd.merge(left=states, right=state_abv, left_on='state', right_on='state')
states.head()




,state,latitude,longitude,abbreviation
0,Alaska,61.3850,-152.2683,AK
1,Alabama,32.7990,-86.8073,AL
2,Arkansas,34.9513,-92.3809,AR
3,Arizona,33.7712,-111.3877,AZ
4,California,36.1700,-119.7462,CA


In [31]:
# states.to_csv("states_info.csv",index=False)

In [7]:

url = "https://covidtracking.com/api/states"

headers = {
 'x-rapidapi-host': 'api host',
 'x-rapidapi-key': 'api token'
 }

response = requests.request("GET", url, headers=headers)
print(response)

<Response [200]>


In [8]:
df = pd.read_json(response.text, orient='records')

df = df.rename(columns={"state":"abbreviation"})


In [9]:
df = pd.merge(left=df, right=states, left_on='abbreviation', right_on='abbreviation')
df.head()



,date,abbreviation,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade,state,latitude,longitude
0,20201205,AK,34963,NaN,1032268,NaN,totalTestsViral,1067231,166.0,799.0,...,d8b27967afb185d8e4f38f5209d713ef23065d88,0,0,0,0,0,,Alaska,61.3850,-152.2683
1,20201205,AL,267589,45785.0,1415357,NaN,totalTestsPeopleViral,1637161,1867.0,26331.0,...,7a521812788c45e903b5a08f357bdd78f668340a,0,0,0,0,0,,Alabama,32.7990,-86.8073
2,20201205,AR,169382,22671.0,1601735,NaN,totalTestsViral,1748446,1056.0,9380.0,...,1570d9633ddb7c4522881fa225446b3967f35981,0,0,0,0,0,,Arkansas,34.9513,-92.3809
3,20201205,AZ,358900,12139.0,2003152,NaN,totalTestsPeopleViral,2349913,2931.0,28006.0,...,b22ef8b238dd16e3fba0bcc29553952e5b2414a4,0,0,0,0,0,,Arizona,33.7712,-111.3877
4,20201205,CA,1311625,NaN,23590350,NaN,totalTestsViral,24901975,10273.0,NaN,...,86d01cdbc69e8f31a43025a1f3f05d3468d5ec15,0,0,0,0,0,,California,36.1700,-119.7462


In [10]:
print(len(df))

50


In [26]:
# Make a data frame with dots to show on the map
data = pd.DataFrame({
   'lat':[-58, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
   'lon':[-34, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
   'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador'],
   'value':[10,12,40,70,23,43,100,43]
})


dataTypeSeries = data.dtypes
print(dataTypeSeries)




lat      float64
lon      float64
name      object
value      int64
dtype: object


In [42]:


# Make an empty map
m = folium.Map(location=[48, -102], zoom_start=3)


# I can add marker one by one on the map
for i in range(0,len(df)):
    folium.Circle(
        location = [df.iloc[i]['latitude'], df.iloc[i]['longitude']],
        popup = str("Positive:") + str(df.iloc[i]['positive']),
        radius = float(df.iloc[i]['positive'])*0.15,
        color = 'crimson',
        fill = True,
        fill_color = 'crimson'
   ).add_to(m)
    


'Positive:35941'

Add a legend to the map using the bronco module and defining the CSS in custom html map elements.

In [43]:

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     width: 160px; height: 100px ; padding: 2px; font-size:14px; right: 2px; bottom: 2px;'>
     
<div class='legend-title'> Positive COVID Cases</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;border-radius:50%;width: 36px;
      height: 36px;'></span>>1000</li>
    <li><span style='background:red;opacity:0.7;border-radius:50%;width: 10px;
      height: 10px;'></span> 100-1000</li>
    <li><span style='background:red;opacity:0.7;border-radius:50%;width: 8px;
      height: 8px;'></span> <100</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m